# Задача мэтчинга товаров

**Цели исследования:**  
  
Для каждого товара магазина найти один или несколько объектов из ассортимента магазина-конкурента, которые близки к нему по некоторой заданной метрике. 

**Задача:**
  
Разработать модель для метчинга товаров в соответствии с требуемой метрикой.
- разработать алгоритм, который для всех товаров магазина предложит несколько вариантов наиболее похожих товаров из ассортимента магазина-конкурента;
- оценить качество алгоритма по метрике accuracy@5

  
**План работы:**
  
- загрузить и изучить представленные данные;
- провести необходимую предобработку данных;
- провести исследовательский анализ данных;
- провести корреляционный анализ признаков, сделать выводы о мультиколлинеарности и при необходимости устранить её.
- выполнить подготовку признаков в пайплайне;
- выбрать лучшую модель и проверить её качество;
- провести анализ важности признаков, сделать выводы об их значимости;
- сформировать выводы и рекомендации по каждому шагу исследования;
- сформировать общий вывод и рекомендации.

**Какими данными располагаем:** 
  
- `base.csv` - анонимизированный набор товаров. Каждый товар представлен как уникальный id (0-base, 1-base, 2-base) и вектор признаков размерностью 72.
- `train.csv` - обучающий датасет. Каждая строчка - один товар, для которого известен уникальный id (0-query, 1-query, …) , вектор признаков И id товара из base.csv, который максимально похож на него (по мнению экспертов).
- `validation.csv` - датасет с товарами (уникальный id и вектор признаков), для которых надо найти наиболее близкие товары из base.csv
- `validation_answer.csv` - правильные ответы к предыдущему файлу.

In [1]:
# FAISS
# Annoy
# Qdrant

In [2]:
%%capture

# Стандартные библиотеки
import os
import re
import sys
import time
import warnings
from datetime import datetime
from math import ceil

# Апдейт и установка необходимых пакетов
!"{sys.executable}" -m pip install -U numba
!"{sys.executable}" -m pip install numpy==1.26.4
!"{sys.executable}" -m pip install scipy==1.13.1
!"{sys.executable}" -m pip install pandas==1.4.4
!"{sys.executable}" -m pip install --upgrade scikit-learn
!"{sys.executable}" -m pip install --upgrade matplotlib
!"{sys.executable}" -m pip install --upgrade seaborn
!"{sys.executable}" -m pip install --upgrade jinja2==3.1.4
!"{sys.executable}" -m pip install catboost
!"{sys.executable}" -m pip install missingno
!"{sys.executable}" -m pip install phik
!"{sys.executable}" -m pip install shap
!"{sys.executable}" -m pip install tqdm 

# Сторонние библиотеки
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import phik
import seaborn as sns
import shap
from IPython.display import display, HTML
from matplotlib.axes._axes import _log as matplotlib_axes_logger
from matplotlib.ticker import MultipleLocator
import missingno as msno
from pandas.plotting import register_matplotlib_converters
from scipy import stats as st

# # Библиотеки scikit-learn
# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.compose import ColumnTransformer
# from sklearn.dummy import DummyRegressor
# from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
# from sklearn.experimental import enable_halving_search_cv
# from sklearn.impute import SimpleImputer
# from sklearn.inspection import permutation_importance
# from sklearn.linear_model import LinearRegression, Lasso, Ridge
# from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
# (
#     GridSearchCV, RandomizedSearchCV, HalvingGridSearchCV, train_test_split
# )
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import (
#     LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder, RobustScaler, StandardScaler
# )
# from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle

# FAISS
import faiss

# Дополнительные библиотеки
from tqdm import tqdm

# Дополнительные настройки
matplotlib_axes_logger.setLevel('ERROR')
warnings.filterwarnings("ignore")
warnings.warn("ignore")
register_matplotlib_converters()

# Зафиксированные параметры визуализации
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
sns.set(rc={'figure.figsize': (20, 10)})
mpl.rcParams.update({'font.size': 11})
sns.set_style("whitegrid")

In [21]:
def unload_df(file_name, parse_dates=None, sep=None, dec=',', index_col=0):
    """
    Ищет файл в сети и локально, загружает его и возвращает как pandas DataFrame.
    Также умеет парсить дату по столбцам.

    :param file_name: имя файла для загрузки
    :param sep: разделитель колонок в файле (например, ',' для CSV)
    :param dec: символ десятичного разделителя (по умолчанию ',')
    :param parse_dates: список столбцов, которые нужно разобрать как даты
    :return: загруженный DataFrame или None, если файл не найден
    """
    if parse_dates is None:
        parse_dates = []
    file_path_net = f'/datasets/{file_name}'
    file_path_local = file_name

    try:
        if os.path.exists(file_path_net):
            file_path = file_path_net
        elif os.path.exists(file_path_local):
            file_path = file_path_local
        else:
            print(f'{file_name} не найден нигде')
            return None
        
        df = pd.read_csv(file_path, parse_dates=parse_dates, sep=sep, decimal=dec, index_col=index_col)
        location = "сети" if file_path == file_path_net else "локального хранилища"
        print(f'{file_name} успешно загружен из {location}')
        return df
    except Exception as e:
        print(f'Произошла ошибка при загрузке: {e}')
        return None
    
def look_on(df):
    display(df.head())
    df.info()

In [22]:
base = unload_df('sample/base.csv')
train = unload_df('sample/train.csv')
validation_answer = unload_df('sample/validation_answer.csv')
validation = unload_df('sample/validation.csv')

sample/base.csv успешно загружен из локального хранилища
sample/train.csv успешно загружен из локального хранилища
sample/validation_answer.csv успешно загружен из локального хранилища
sample/validation.csv успешно загружен из локального хранилища


In [23]:
look_on(base)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4207931-base,-43.946243,15.364378,17.515854,-132.31146,157.064420,-4.069252,-340.630860,-57.550140,128.398220,45.090958,-126.84374,4.494522,-99.842310,44.926903,177.52173,-12.291790,38.470360,105.357650,-142.46024,-80.163260,-110.368935,1047.517357,-69.594620,66.313540,84.873870,813.770071,-81.038780,16.162964,-98.244880,159.53406,27.554913,-209.184280,62.05977,-529.295053,114.598330,90.469894,-20.256914,-164.76800,-133.31387,-41.252960,-10.251193,8.289038,-131.312710,75.704500,-16.483078,40.771038,-146.096740,-143.407680,49.807987,63.434480,-30.250080,20.470263,78.079910,-128.915310,92.327680,63.885570,-141.17464,142.902590,-93.068596,-568.421584,-90.018690,-129.015670,-71.927170,30.711966,-90.190475,-24.931271,66.972534,106.346634,-44.270622,155.98834,-1074.464888,-25.066608
2710972-base,-73.004890,4.923342,-19.750746,-136.52908,99.907170,-70.709110,-567.401996,-128.890150,109.914986,201.472200,-186.22650,29.896042,-99.770996,0.126302,136.19049,-35.224740,-30.321323,-43.148834,-162.85175,-79.714510,-75.784870,1507.231274,-69.654564,43.640663,-4.779669,813.770071,43.976913,11.924875,-50.228523,166.00820,-59.505333,-115.332520,72.18324,-735.671365,96.322300,85.796360,-22.030330,-147.54501,-108.38295,-45.084892,-15.004004,-1.532826,-46.456585,197.578950,-56.199876,60.298710,-102.653340,-108.967964,58.512012,-9.678028,-85.448300,-68.686080,71.590200,-232.425690,91.706856,63.290657,-137.33595,-47.124687,-148.057400,-543.787056,-160.651600,-133.462220,-109.044660,20.916021,-171.201390,-110.596844,67.730100,8.909615,-9.470253,133.29536,-545.897014,-72.913230
1371460-base,-85.565570,-0.493598,-48.374817,-157.98502,96.809510,-81.710210,-22.297688,79.768670,124.357086,105.715180,-149.80756,-54.501680,-21.037973,-24.887660,128.38864,-58.558483,34.862656,19.784412,-130.91820,-79.032230,-166.635250,1507.231274,-8.495993,61.205086,25.895348,813.770071,-140.768860,20.872790,-123.957570,126.34781,11.713674,-125.025154,152.68590,-1018.469545,-22.444600,73.897640,9.190645,-156.51881,-92.18573,-34.926760,-13.277475,16.026424,-33.853546,119.604520,-52.525341,71.204750,-178.702940,-88.278500,30.501453,16.651737,-88.377014,-55.883583,70.182980,-89.233925,92.005780,76.458725,-131.14087,40.914352,-157.900540,-394.319235,-87.107025,-120.772545,-58.821650,41.369606,-132.934500,-43.016839,67.871925,141.778240,69.048520,111.72038,-1111.038833,-23.087206
3438601-base,-105.564090,15.393871,-46.223934,-158.11488,79.514114,-48.944480,-93.713010,38.581398,123.397960,110.324326,-161.18800,-68.519790,-0.607330,38.733696,120.74344,-14.109269,28.868027,-29.858810,-94.30395,-79.339810,-138.984270,1507.231274,-131.885380,70.031360,32.736595,813.770071,-62.370860,13.763219,-31.872276,139.55270,9.836465,-150.221130,80.14020,-537.183707,3.091667,129.699330,-63.429424,-169.02724,-119.77007,-28.637785,-8.315162,2.752385,-160.293820,85.086890,-18.251750,90.374054,1.479935,-121.983050,65.852660,8.355225,34.118896,-57.069756,70.461800,-127.905410,94.314280,71.259940,-135.57787,-39.982346,-159.751560,-230.147648,-95.221160,-148.814090,-87.907290,-58.806870,-147.794800,-155.830237,68.974754,21.397510,126.098785,139.73320,-1282.707248,-74.527940
422798-base,-74.638880,11.315012,-40.204174,-161.76430,50.507114,-80.775560,-640.923467,65.225000,122.344940,191.465850,-156.98384,-76.650210,-75.674970,12.624029,145.33752,-35.774258,11.598761,-11.460761,-201.35443,-77.779366,-120.968400,548.736883,19.851685,17.943344,27.063320,813.770071,-85.483780,21.236433,-95.071020,132.61092,13.526038,-160.476840,104.71937,-304.174382,-15.385452,91.418655,-36.474556,-157.43959,-102.83162,-56.782710,-19.969252,-0.598189,-222.228790,33.441666,-56.092110,71.276030,-8.713509,-86.099380,8.488903,-14.959278,86.812996,-29.6

<class 'pandas.core.frame.DataFrame'>
Index: 291813 entries, 4207931-base to 274130-base
Data columns (total 72 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   0       291813 non-null  float64
 1   1       291813 non-null  float64
 2   2       291813 non-null  float64
 3   3       291813 non-null  float64
 4   4       291813 non-null  float64
 5   5       291813 non-null  float64
 6   6       291813 non-null  float64
 7   7       291813 non-null  float64
 8   8       291813 non-null  float64
 9   9       291813 non-null  float64
 10  10      291813 non-null  float64
 11  11      291813 non-null  float64
 12  12      291813 non-null  float64
 13  13      291813 non-null  float64
 14  14      291813 non-null  float64
 15  15      291813 non-null  float64
 16  16      291813 non-null  float64
 17  17      291813 non-null  float64
 18  18      291813 non-null  float64
 19  19      291813 non-null  float64
 20  20      291813 non-null  float64
 21 

In [24]:
look_on(train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,Target
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
109249-query,-24.021454,3.122524,-80.947525,-112.329994,191.090180,-66.903130,-759.626065,-75.284454,120.55149,131.131700,-149.211060,-102.31221,21.387623,11.277594,143.22140,-22.011570,-3.618249,-16.005480,-133.38228,-78.89356,-65.690530,407.773575,-11.660624,67.008150,24.975033,813.770071,40.051064,17.933155,-75.435745,149.81720,-23.413877,-178.095570,133.786470,-906.571061,113.355560,83.942260,-16.592659,-146.52074,-120.237860,-27.341612,-8.845615,1.027612,-175.647720,167.735820,-32.931559,47.860960,-196.247500,-118.81005,-4.762772,-114.877680,37.397278,-55.616966,56.627056,-108.43317,87.372560,76.513430,-136.27057,3.652915,-164.574510,-635.284275,-75.647255,-116.679340,-41.234684,-24.601670,-167.760770,133.678516,68.184600,26.317545,11.938202,148.549320,-778.563381,-46.877750,66971-base
34137-query,-82.033580,8.115866,-8.793022,-182.972100,56.645336,-52.597610,-55.720337,130.059250,129.38335,76.202880,-137.799420,33.30165,-2.868191,-34.318770,189.06479,-19.337550,-14.208210,-71.110245,-157.74814,-78.70069,-91.741875,1054.205600,-41.845630,102.128620,72.559050,813.770071,-37.957787,17.598982,-159.975400,140.02528,-8.819328,-147.055180,113.819870,-529.295053,70.674940,55.976795,8.817799,-134.14812,-73.679794,-57.566544,-4.338496,-3.270682,-144.499200,144.650200,-37.903276,58.913525,-105.362840,-125.66783,19.367283,-29.087658,-35.021350,26.627962,55.718437,-110.52611,83.513374,75.926130,-135.68242,-7.429803,-180.645020,11.470171,16.464691,-121.807236,-90.814450,54.448433,-120.894806,-12.292085,66.608116,-27.997612,10.091335,95.809265,-1022.691531,-88.564705,1433819-base
136121-query,-75.719640,-0.223386,-86.186130,-162.064060,114.320114,-53.394600,-117.261013,-24.857851,124.80780,112.190155,-200.925960,-38.86518,-80.611270,14.343805,156.62129,-22.498169,-26.359468,-109.034870,-106.92659,-79.74731,-69.876830,1507.231274,-20.058287,34.334927,23.592144,813.770071,-49.503860,22.166200,-85.740160,134.83647,-69.569850,-139.887240,67.377045,-341.781842,54.161224,81.891660,36.421352,-159.99583,-131.916080,-20.495195,-13.976569,-2.355247,-216.228650,238.836490,-56.611536,43.366640,7.191841,-159.48369,-19.338009,-51.409897,36.819540,32.536880,80.681020,-232.40741,84.053690,59.086180,-139.85950,78.409440,-115.940575,2.426572,7.594826,-126.520134,-73.148960,-5.609123,-93.029880,-80.997871,63.733383,11.378683,62.932007,130.975390,-1074.464888,-74.861176,290133-base
105191-query,-56.580620,5.093593,-46.943110,-149.039120,112.436430,-76.820510,-324.995645,-32.833107,119.47865,120.074790,-61.347084,-28.67060,-102.790180,-36.194320,157.18976,-33.318240,7.448413,-47.230713,-178.04608,-78.78652,-106.235440,1507.231274,-63.414307,38.099255,-89.795350,813.770071,-107.432390,10.052701,-71.917380,147.74005,-18.750763,-143.795620,67.207310,-366.139446,112.187700,78.144810,-41.085410,-132.75719,-89.445030,-19.267069,-14.866466,7.775788,-104.302110,74.622894,-59.875136,76.406470,-77.797020,-92.01658,19.337300,-37.922787,37.271270,111.639570,94.912950,-179.72540,86.601480,62.698364,-122.16293,29.873940,-53.508120,-0.938894,-36.919907,-144.555000,-96.798590,21.624313,-158.880370,179.597294,69.891360,-33.804955,233.914610,122.868546,-1074.464888,-93.775375,1270048-base
63983-query,-52.725650,9.027046,-92.829650,-113.111010,134.124970,-42.423073,-759.626065,8.261169,119.49023,172.365360,-186.641390,-84.94380,-92.339966,-30.229528,167.86163,-22.635653,0.014536,-9.796367,-213.10180,-78.59006,-98.728300,1250.423749,-43.892487,86.288450,-1.549826,813.770071,-110.356980,24.055641,-96.578270,156.58230,45.124240,-123.888504,118.035110,-607.946912,52.311410,76.747800,-14.161914,-143.53851,-124.886215,-64.783330,-17.706848,15.446568,-53.554455,174.381620,-23.140892,76.419330,-73.

<class 'pandas.core.frame.DataFrame'>
Index: 9999 entries, 109249-query to 13504-query
Data columns (total 73 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       9999 non-null   float64
 1   1       9999 non-null   float64
 2   2       9999 non-null   float64
 3   3       9999 non-null   float64
 4   4       9999 non-null   float64
 5   5       9999 non-null   float64
 6   6       9999 non-null   float64
 7   7       9999 non-null   float64
 8   8       9999 non-null   float64
 9   9       9999 non-null   float64
 10  10      9999 non-null   float64
 11  11      9999 non-null   float64
 12  12      9999 non-null   float64
 13  13      9999 non-null   float64
 14  14      9999 non-null   float64
 15  15      9999 non-null   float64
 16  16      9999 non-null   float64
 17  17      9999 non-null   float64
 18  18      9999 non-null   float64
 19  19      9999 non-null   float64
 20  20      9999 non-null   float64
 21  21      9999 non-null   

In [25]:
look_on(validation)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71
Id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
196680-query,-59.383420,8.563436,-28.203072,-134.22534,82.73661,-150.572170,-129.178969,23.670555,125.66636,108.809586,-129.48387,-178.983060,-109.600174,-8.799808,172.95998,-20.794373,-30.065893,-14.889741,-213.47429,-81.442860,-92.558720,1507.231274,96.508420,87.975250,55.862797,813.770071,1.647972,16.160482,-77.401474,166.08685,-7.085945,-114.40581,116.564270,-481.586956,40.185913,73.085365,-37.582203,-140.10822,-113.26041,-64.863230,-16.001427,7.223721,-5.791832,154.656310,-34.690983,52.748238,-34.976818,-160.459520,-28.526081,11.436787,107.386640,33.117570,56.67899,-43.842407,95.18327,51.950043,-123.31064,-10.645209,-52.291348,-525.623407,53.718872,-129.388460,-103.481630,79.564530,-120.31357,54.218155,68.50073,32.681908,84.19686,136.412960,-1074.464888,-21.233612
134615-query,-103.912150,9.742726,-15.209915,-116.37310,137.69880,-85.530075,-776.123158,44.481530,114.67121,95.231290,-166.03618,-66.359830,-36.001366,3.264235,73.06930,-29.384926,22.245693,62.498410,-114.18031,-80.017426,-56.034016,914.812090,-23.072426,64.591540,47.074090,813.770071,1.761437,24.459257,-177.638370,157.88023,-15.648800,-174.11716,37.697598,-701.605866,18.383450,81.502020,22.231460,-129.41878,-117.69812,-53.364460,-4.394635,11.108950,-109.880050,102.263280,-47.268603,52.336370,31.617912,13.088348,0.388435,-55.594444,-37.935482,-46.970780,50.48210,-132.518330,88.67881,81.240204,-130.75761,4.710941,-114.013050,-433.616738,-119.455990,-129.188340,-51.193770,49.299644,-101.89454,105.560548,67.80104,13.633057,108.05138,111.864456,-841.022331,-76.567980
82675-query,-117.923280,-3.504554,-64.299390,-155.18713,156.82137,-34.082264,-537.423653,54.078613,121.97396,59.321335,-90.08289,4.986931,-52.514560,52.529945,140.47353,-4.860558,-18.063830,-36.537400,-137.92374,-79.661070,-70.733120,1507.231274,-7.057582,26.213560,-2.779066,813.770071,-69.704410,16.080505,-90.432610,137.94106,24.971474,-138.86641,92.287190,-735.671365,68.335190,78.208220,14.043610,-147.51697,-113.89963,-18.748684,-8.779379,-8.737224,-177.382870,156.102450,-35.756027,65.317690,-262.907840,-96.018070,55.713432,22.165249,151.100540,-24.815138,70.92110,-121.119310,91.86982,87.153366,-138.07550,-3.309690,14.035965,-107.596636,-152.853940,-118.997840,-115.176155,48.636130,-132.17967,-0.988696,68.11125,107.065216,134.61765,134.080000,27.773269,-32.401714
162076-query,-90.880554,4.888542,-39.647797,-131.75010,62.36212,-105.593270,-347.132493,-83.351750,133.91331,201.146090,-193.19345,-31.961876,-11.191006,-28.481222,157.13997,-39.513940,-20.431585,30.671173,-131.63226,-79.841600,-74.222690,1507.231274,-75.135840,34.678430,-14.997078,401.379624,-29.014805,17.788988,-87.424790,160.81638,-13.624538,-137.01877,89.403885,-388.662473,-0.446587,73.493530,3.995680,-144.55515,-125.87352,-35.733467,-9.979044,2.092319,-114.457405,158.609240,-58.275016,96.416830,-166.106690,-36.610770,95.944460,-43.662690,33.869110,30.895940,65.87759,-106.503220,94.52601,72.289566,-152.20987,29.090012,-188.342150,-327.117943,43.212470,-139.852200,-112.293790,54.884007,-177.56935,-116.374997,67.88766,136.893980,124.89447,117.707750,-566.343980,-90.905556
23069-query,-66.946740,10.562773,-73.781830,-149.39787,2.93866,-51.288853,-587.189361,-2.764402,126.56105,131.900620,-131.93640,-35.794685,-155.979580,-2.110109,137.72418,-11.544052,-12.957520,2.028175,-129.12962,-79.461266,-72.883120,1507.231274,-47.112600,7.837235,-8.623394,813.770071,-60.251694,11.591301,-82.794800,134.84439,4.764982,-114.47928,83.150400,-156.249890,41.852833,42.160450,-58.565960,-146.39613,-90.59503,-53.295376,-12.213371,-8.682546,-142.693270,71.629135,-57.668621,55.122387,10.182793,-100.190810,-45.052837,-46.877544,10.418076,106.135445,75.15257,-110.051500,83.05377,97.87521

<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 196680-query to 43566-query
Data columns (total 72 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       10000 non-null  float64
 1   1       10000 non-null  float64
 2   2       10000 non-null  float64
 3   3       10000 non-null  float64
 4   4       10000 non-null  float64
 5   5       10000 non-null  float64
 6   6       10000 non-null  float64
 7   7       10000 non-null  float64
 8   8       10000 non-null  float64
 9   9       10000 non-null  float64
 10  10      10000 non-null  float64
 11  11      10000 non-null  float64
 12  12      10000 non-null  float64
 13  13      10000 non-null  float64
 14  14      10000 non-null  float64
 15  15      10000 non-null  float64
 16  16      10000 non-null  float64
 17  17      10000 non-null  float64
 18  18      10000 non-null  float64
 19  19      10000 non-null  float64
 20  20      10000 non-null  float64
 21  21      10000 non-null 

In [26]:
look_on(validation_answer)

,Expected
Id,
196680-query,1087368-base
134615-query,849674-base
82675-query,4183486-base
162076-query,2879258-base
23069-query,615229-base


<class 'pandas.core.frame.DataFrame'>
Index: 10000 entries, 196680-query to 43566-query
Data columns (total 1 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Expected  10000 non-null  object
dtypes: object(1)
memory usage: 156.2+ KB


In [27]:
train.Target.value_counts()

803095-base     3
804781-base     3
272609-base     3
839040-base     3
79670-base      3
               ..
2222522-base    1
24444-base      1
501422-base     1
826204-base     1
505720-base     1
Name: Target, Length: 9672, dtype: int64

In [28]:
12
34
56

56

## 2.3 EDA

Несмотря на то, что данные обезличены, EDA здесь также будет полезен: все ли столбцы имеют одинаковое распределение значений? Есть ли столбцы, которые для модели были бы мало полезны? Есть ли сильно скоррелированные друг с другом столбцы? Может быть, есть смысл на первом этапе подавать в модель не все фичи, а наиболее информативные? Есть ли пропуски? Явные дубликаты? Если есть - что с ними делать? Есть ли аномалии в распределениях? Следующий важный вопрос - не требуется ли масштабирование данных? Ответить на этот вопрос можно, например, замерив метрику с масштабированием и без масштабирования признаков.

## 2.4 Целевая метрика

Наша целевая метрика - accuracy@n. Собственно, что это такое. Вспомним, что 

$$
Accuracy = \frac{Correct\ predictions}{All\ predictions}
$$

Представим расчет метрики в цикле, перебирая все предложенные моделью ответы. При этом каждое предсказание содержит в себе не 1 ответ, а сразу n, и если среди предложенных вариантов окажется правильный - числитель и знаменатель увеличиваются на 1. А если нет ни одного - то на 1 увеличивается только знаменатель. В нашей задаче n = 5. Хорошо бы добиться accuracy@5 ≥ 0,7. Кстати, легко заметить, что accuracy@1 - это самая обычная accuracy.

## 📊 Create FAISS [index](https://github.com/facebookresearch/faiss/wiki/Faiss-indexes) for small dataset


[Guideline](https://github.com/facebookresearch/faiss/wiki/Guidelines-to-choose-an-index)

Hint: Use numpy [ascontigiousarray](https://numpy.org/doc/stable/reference/generated/numpy.ascontiguousarray.html) - object which is stored in one [unbroken block](https://www.educative.io/answers/what-is-the-numpyascontiguousarray-function-in-python) in memory -  to load vectors in FAISS

In [29]:
dims = base.shape[1]
n_cells = 20
quantizer = faiss.IndexFlatL2(dims)
idx_l2 = faiss.IndexIVFFlat(quantizer, dims, n_cells)

In [30]:
%%time
idx_l2.train(np.ascontiguousarray(base.values).astype('float32'))
idx_l2.add(np.ascontiguousarray(base.values).astype('float32'))

CPU times: user 178 ms, sys: 107 ms, total: 285 ms
Wall time: 207 ms


In [31]:
base_index = {k: v for k, v in enumerate(base.index.to_list())}

## 🔍 Search

In [32]:
targets = train["Target"]
train.drop("Target", axis=1, inplace=True)

In [33]:
%%time
candidate_number = 5
r, idx = idx_l2.search(np.ascontiguousarray(train.values).astype('float32'), candidate_number)

CPU times: user 2.34 s, sys: 17 ms, total: 2.36 s
Wall time: 325 ms


## 📈 Accuracy@candidate_number calculation

In [34]:
acc = 0
for target, el in zip(targets.values.tolist(), idx.tolist()):
    acc += int(target in [base_index[r] for r in el])
print(f'Accuracy @ {candidate_number} = {acc / len(idx):.1%}')

Accuracy @ 5 = 11.5%


In [35]:
## ❓❓❓ What's next?

For full dataset it is strongly recommended to test your code on the small batch before loading all dataset to FAISS

You can make your own research:
- change number of cells
- change number of candidates
- change indexes
- add another ML models to improve the FAISS result
- change the accelerator: Hint: Search method on GPU differs a bit from the similar method on CPU
-.....

Remember, that in Colab you have only 12 GB of RAM, so remove variables and objects if necessary

**Good Luck!**

SyntaxError: invalid syntax (2633361802.py, line 3)